In [ ]:
# !pip install -r requirements.txt

In [25]:
pip install pprint

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [26]:
import dotenv
import os
import openai
import json
import pandas as pd
import sys
import requests
import re
import pprint
# .env 파일 로드 (있다면)
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
SLACK_URL = os.getenv("SLACK_URL")

In [4]:
# 작업 종료 후 slack으로 메신저 보내기기
def send_msg(msg):
    url = SLACK_URL
    message = (msg + " 작업 완료") 
    title = (f"New Incoming Message :zap:") # 타이틀 입력
    slack_data = {
        "username": "NotificationBot", # 보내는 사람 이름
        "icon_emoji": ":satellite:",
        #"channel" : "#somerandomcahnnel",
        "attachments": [
            {
                "color": "#9733EE",
                "fields": [
                    {
                        "title": title,
                        "value": message,
                        "short": "false",
                    }
                ]
            }
        ]
    }
    byte_length = str(sys.getsizeof(slack_data))
    headers = {'Content-Type': "application/json", 'Content-Length': byte_length}
    response = requests.post(url, data=json.dumps(slack_data), headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

In [ ]:
# 내용 받았다고 가정
## 나중에 백엔드에서 requests.get 형식으로 내용 받아서 작성 필요
### 여기선 테스트로 1개의 데이터로만 진행
df = pd.read_csv("data_detail.csv")

user_df = pd.read_csv("data_user_detail_plus.csv")

user_data = user_df['user_data'].tolist()[0]
text = df['text'].to_list()[0]
content = df['detail'].to_list()[0]

In [41]:
print(user_data)

[{'projectName': 'StudyHub', 'projectTarget': '대학생 및 자격증 준비생', 'mainFunction': ['스터디 그룹 생성 및 관리', '퀴즈 생성 및 참여 기능', '공지사항 및 과제 게시판'], 'techStack': ['Java Servlet', 'JSP', 'MySQL'], 'projectDescription': 'GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹 관리에서 나아가 그룹 내 학습 내용을 복습할 수 있는 기능을 제공하여 학습 효과를 극대화하는 것을 목표로 한다. 사용자들은 회원가입과 로그인을 통해 서비스를 이용할 수 있으며, 아이디 및 비밀번호 찾기 기능으로 편의성을 높였다. 사용자는 새로운 스터디 그룹을 생성하거나 기존 그룹을 검색해 가입할 수 있으며, 그룹 내에서는 퀴즈를 생성하고 참여하여 학습한 내용을 효과적으로 반복 학습할 수 있다. 또한 공지사항 게시판을 통해 그룹 내 전달사항을 공유할 수 있고, 과제 게시판은 체크리스트 형태로 구성되어 과제 수행 여부를 명확히 기록할 수 있다. 이러한 기능들은 사용자 간의 소통을 강화하고 학습 동기를 부여하는 데 기여한다. 서비스는 Java Servlet과 JSP 기반으로 구현되었으며, HTML, CSS, JavaScript를 통해 사용자 친화적인 인터페이스를 제공한다. 데이터베이스는 MySQL을 사용하여 안정적인 데이터 저장과 처리를 가능하게 하며, 전반적인 시스템은 학습 관리에 최적화된 구조로 설계되었다.'}]


In [42]:
print(content)

[{'requirementType': 'FUNCTIONAL1', 'content': '사용자는 회원가입 시 이메일 인증을 통해 계정을 활성화할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL2', 'content': '사용자는 스터디 그룹 내에서 퀴즈를 생성하고, 생성된 퀴즈에 대해 다른 사용자들이 참여할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL3', 'content': '사용자는 과제 게시판에서 각 과제의 수행 여부를 체크리스트 형태로 기록하고 관리할 수 있어야 한다.'}, {'requirementType': 'PERFORMANCE1', 'content': '시스템은 사용자 요청에 대한 응답을 평균 2초 이내에 처리해야 한다.'}, {'requirementType': 'PERFORMANCE2', 'content': '시스템은 동시에 최대 100명의 사용자가 접속하더라도 95% 이상의 가용성을 유지해야 한다.'}, {'requirementType': 'PERFORMANCE3', 'content': '데이터베이스는 1초 이내에 1000건의 퀴즈 데이터를 검색할 수 있어야 한다.'}]


In [ ]:
client = openai.OpenAI()

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 소프트웨어 프로젝트의 요구사항 분석 전문가입니다.
주어진 프로젝트 개요를 바탕으로 현실적이고 구체적인 기능 요구사항과 성능 요구사항을 생성하는 것이 목표입니다.

생성 규칙:
1. 기능 요구사항(FUNCTIONAL): 사용자 관점에서 시스템이 제공해야 하는 구체적인 기능
2. 성능 요구사항(PERFORMANCE): 시스템의 응답시간, 처리량, 가용성, 확장성 등 측정 가능한 수치 포함
3. 각 요구사항은 명확하고 구현 가능하며 테스트 가능해야 함
4. 프로젝트 규모와 성격에 적합한 현실적인 수준이어야 함
5. 반드시 지정된 JSON 형식으로만 응답할 것
"""

# 파인튜닝된 모델 ID
MODEL_ID = 'gpt-4o-mini'

def detail_plus(project_overview,existing_requirements, additional_count = 5):
  # 구조화된 요청 프롬프트 (f-string 사용하지 않음)
    enhanced_prompt = f"""

    프로젝트 주제 및 입력사항:
    {project_overview}

    기존 요구사항 목록:
    {existing_requirements}

    위 기존 주제와 요구사항들을 참고하여 추가 요구사항을 생성해주세요
    또한 이렇게 생성된 요구사항은 원래의 **'기존 요구사항 목록'**에 추가시켜서 정렬 후 출력하세요:

    **생성 조건:**
    - 기존 주제와 요구사항과 일관성을 유지하면서 새로운 {additional_count}개의 요구사항 생성
    - 기존 주제와 요구사항의 도메인과 맥락을 고려
    - 기능 주제와 요구사항과 성능 요구사항을 적절히 혼합
    - 기존 요구사항과 중복되지 않는 새로운 관점의 요구사항

    **중요: 반드시 아래 JSON 형식으로만 응답하세요. 다른 텍스트, 설명, 주석은 절대 포함하지 마세요.**

    JSON 형식:
    [
      {{"requirementType": "FUNCTIONAL1", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "FUNCTIONAL2", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "FUNCTIONAL3", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "PERFORMANCE2", "content": "구체적인 성능 요구사항 (수치 포함)"}},
      {{"requirementType": "PERFORMANCE1", "content": "구체적인 성능 요구사항 (수치 포함)"}}
    ]

    응답은 반드시 위 JSON 배열로만 시작하고 끝나야 합니다.
    """
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"오류 발생: {e}"

In [49]:
result = json.loads(detail_plus(user_data, content))
print(result)
send_msg("LLM")

[{'requirementType': 'FUNCTIONAL1', 'content': '사용자는 회원가입 시 이메일 인증을 통해 계정을 활성화할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL2', 'content': '사용자는 스터디 그룹 내에서 퀴즈를 생성하고, 생성된 퀴즈에 대해 다른 사용자들이 참여할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL3', 'content': '사용자는 과제 게시판에서 각 과제의 수행 여부를 체크리스트 형태로 기록하고 관리할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL4', 'content': '사용자는 스터디 그룹 내에서 공지사항을 작성하고, 다른 사용자들이 이를 열람할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL5', 'content': '사용자는 자신의 프로필을 수정하고, 프로필 사진을 업로드할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL6', 'content': '사용자는 퀴즈 참여 후 결과를 확인하고, 자신의 성적을 기록할 수 있어야 한다.'}, {'requirementType': 'PERFORMANCE1', 'content': '시스템은 사용자 요청에 대한 응답을 평균 2초 이내에 처리해야 한다.'}, {'requirementType': 'PERFORMANCE2', 'content': '시스템은 동시에 최대 100명의 사용자가 접속하더라도 95% 이상의 가용성을 유지해야 한다.'}, {'requirementType': 'PERFORMANCE3', 'content': '데이터베이스는 1초 이내에 1000건의 퀴즈 데이터를 검색할 수 있어야 한다.'}, {'requirementType': 'PERFORMANCE4', 'content': '시스템은 1분당 최대 500건의 퀴즈 생성 요청을 처리할 수 있어야 한다.'}, {'requirementType'

In [1]:
!pip install -r requirements.txt

     ---------------------------------------- 0.0/174.6 kB ? eta -:--:--
     -------------------------------------- 174.6/174.6 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.

The conflict is caused by:
    langchain-community 0.0.38 depends on langsmith<0.2.0 and >=0.1.0
    langchain 0.0.352 depends on langsmith<0.1.0 and >=0.0.70

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict



ERROR: Cannot install -r requirements.txt (line 3) and -r requirements.txt (line 4) because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
